In [55]:
from helpers import *
import scraper_functions
import pandas as pd
import datetime

import os
from __init__ import const_datasetLibDir
#import __init__

In [56]:
def get_stocks_earnings_prices(startDate, endDate, returnDataFrames = True, env = 'dev', intervalMins=30, periodDays=90):
    '''Scrape earnings data from yahoo then scrape the price data & stock summary afterwards.'''
    
    libraryDir = os.path.join(const_datasetLibDir, env)
    #declare a process log for capturing success and failed data
    processLog = pd.DataFrame(columns = ['dataset','outcome','id'])
    
    #scrape earnings data
    earnings = scraper_functions.earnings_yahoo_range(startDate,endDate)
    
    #grab the list of symbols
    #symbols = pd.read_csv(libraryDir + 'symbols.csv', sep='\t',index_col=0)
    symbols = pd.DataFrame(earnings['Symbol'])
    
    #return the prices for the list of symbols
    prices, success, fail = scraper_functions.prices_google_symbols(symbols)
    processLog = processLog.append(pd.DataFrame({'dataset': 'earnings', 'outcome': 'success', 'id': success}))
    processLog = processLog.append(pd.DataFrame({'dataset': 'earnings', 'outcome': 'fail', 'id': fail}))

    #return the stock summary data (and success/failed symbols)
    stocksummary, success, fail = scraper_functions.stocksummary_yahoo_symbols(symbols)
    processLog = processLog.append(pd.DataFrame({'dataset': 'stocksummary', 'outcome': 'success', 'id': success}))
    processLog = processLog.append(pd.DataFrame({'dataset': 'stocksummary', 'outcome': 'fail', 'id': fail}))
    
    print 'Saving files to: ' + libraryDir
    
    #determine the library name and the directory
    now = datetime.datetime.now().strftime("%Y-%m-%d %H%M%S")
    libraryName = 'stocks_earnings_prices_' + str(intervalMins) + 'm_' + startDate + '_' + endDate + '_' + now
    libraryDir = os.path.join(libraryDir, libraryName)
    
    #check if the directory exists / make a new one if doesnt
    checkDirExists(libraryDir, make=True)
    
    #store results as csv's
        
    earnings.to_csv(os.path.join(libraryDir,'earnings.csv'), sep='\t')
    earnings['Symbol'].to_csv(os.path.join(libraryDir, 'symbols.csv'), sep='\t', header=True)
    prices.to_csv(os.path.join(libraryDir, 'prices.csv'), sep='\t')
    stocksummary.to_csv(os.path.join(libraryDir, 'stocksummary.csv'), sep='\t')
    processLog.to_csv(os.path.join(libraryDir, 'processLog.csv'), sep='\t')
    
    return earnings, symbols, prices, stocksummary, processLog